In [1]:
import pandas as pd
import os, errno
import time
import itertools

In [2]:
meta = pd.read_csv('vox1_meta.csv', sep='\t')

In [4]:
print(meta["Nationality"].value_counts())
print(meta["Gender"].value_counts())

USA                    799
UK                     215
Canada                  54
Australia               37
India                   26
Norway                  20
Ireland                 18
Germany                  9
New Zealand              8
Italy                    8
Mexico                   7
Sweden                   5
Russia                   4
Spain                    4
Philippines              3
Denmark                  3
Chile                    3
Switzerland              3
Croatia                  3
Netherlands              3
Poland                   2
China                    2
Portugal                 2
Guyana                   1
Sri Lanka                1
France                   1
Israel                   1
Brazil                   1
Trinidad and Tobago      1
Singapore                1
South Korea              1
Sudan                    1
Austria                  1
Pakistan                 1
South Africa             1
Iran                     1
Name: Nationality, dtype: in

In [4]:
meta

,VoxCeleb1 ID,VGGFace1 ID,Gender,Nationality,Set
0,id10001,A.J._Buckley,m,Ireland,dev
1,id10002,A.R._Rahman,m,India,dev
2,id10003,Aamir_Khan,m,India,dev
3,id10004,Aaron_Tveit,m,USA,dev
4,id10005,Aaron_Yoo,m,USA,dev
...,...,...,...,...,...
1246,id11247,Zachary_Levi,m,USA,dev
1247,id11248,Zachary_Quinto,m,USA,dev
1248,id11249,Zack_Snyder,m,USA,dev
1249,id11250,Zoe_Saldana,f,USA,dev


In [5]:
def GetIds(male=True):
    g = 'm' if male else 'f'
    return meta[meta["Gender"] == g]['VoxCeleb1 ID']

def get_intersection_combos(sensitive_param_value_groups):
    list(itertools.product(a, b))

In [6]:
gender_keys = ['m', 'f']
# @TODO Add all countries
country_keys = ['USA', 'UK', 'Canada']


male_ids = set(GetIds())
female_ids = set(GetIds(male=False))

In [7]:
USA_ids = meta[meta["Nationality"] == 'USA']['VoxCeleb1 ID']
UK_ids = meta[meta["Nationality"] == 'UK']['VoxCeleb1 ID']
Canada_ids = meta[meta["Nationality"] == 'Canada']['VoxCeleb1 ID']
Australia_ids = meta[meta["Nationality"] == 'Australia']['VoxCeleb1 ID']
India_ids = meta[meta["Nationality"] == 'India']['VoxCeleb1 ID']
Norway_ids = meta[meta["Nationality"] == 'Norway']['VoxCeleb1 ID']
Ireland_ids = meta[meta["Nationality"] == 'Ireland']['VoxCeleb1 ID']

In [8]:
utters = pd.read_csv('/home/jupyter/voxceleb-fairness/data/datasets/full/vox1_full_utterances.txt', header=None, sep=' ')

In [9]:
male_utters = utters[[x in male_ids for x in utters[0]]].sample(5000).reset_index(drop=True)

In [10]:
male_utters

,0,1
0,id10227,id10227/is4k-QF2Y1I/00001.wav
1,id10045,id10045/wUVMEE3Y32A/00003.wav
2,id10454,id10454/q0WkZ5RKo_4/00005.wav
3,id10505,id10505/nkzVdeDZG2E/00011.wav
4,id10243,id10243/eCwriPR_VdU/00004.wav
...,...,...
4995,id10297,id10297/o1EKUvjvNHY/00009.wav
4996,id10363,id10363/oOER7ITQrjA/00006.wav
4997,id10489,id10489/2712Ba9UPMk/00005.wav
4998,id10636,id10636/HCjBxAD36FY/00005.wav


In [11]:
female_utters = utters[[x in female_ids for x in utters[0]]].sample(5000).reset_index(drop=True)

In [12]:
def silentremove(filename):
    try:
        os.remove(filename)
    except OSError as e: # this would be "except OSError, e:" before Python 2.6
        if e.errno != errno.ENOENT: # errno.ENOENT = no such file or directory
            raise # re-raise exception if a different error occurred

In [13]:
def make_pairs(utters_df, filename):
    silentremove(filename)
    start = time.time()
    current_iter_start = start
    for i in range(len(utters_df)):
        data = []
        for j in range(i + 1, len(utters_df)):
            num = 1 if utters_df[0][i] == utters_df[0][j] else 0
            data.append([num, utters_df[1][i], utters_df[1][j]])
        pd.DataFrame(data).to_csv(filename, mode='a', index=False, header=None, sep=' ')
        if i % 1000 == 0:
            current_iter_end = time.time()
            print('Wrote {} of {} utterances in {} seconds ({} seconds from start)'.format(i, len(utters_df), current_iter_end - current_iter_start, current_iter_end - start))
            current_iter_start = current_iter_end

In [14]:
make_pairs(male_utters, '/home/jupyter/voxceleb-fairness/data/lists/vox1_male_all.txt')

Wrote 0 of 5000 utterances in 0.1301438808441162 seconds (0.1301438808441162 seconds from start)


KeyboardInterrupt: 

In [ ]:
male_pairs = pd.read_csv('/home/jupyter/voxceleb-fairness/data/lists/vox1_male_all.txt', header=None, sep=' ')

In [ ]:
balanced_male_pairs = pd.concat([male_pairs[male_pairs[0] == 1].reset_index(drop=True), male_pairs[male_pairs[0] == 0].sample(sum(male_pairs[0] == 1)).reset_index(drop=True)]).sort_index().reset_index(drop=True)

In [ ]:
balanced_male_pairs

In [ ]:
make_pairs(female_utters, '/home/jupyter/voxceleb-fairness/data/lists/vox1_female_all.txt')

In [ ]:
female_pairs = pd.read_csv('/home/jupyter/voxceleb-fairness/data/lists/vox1_female_all.txt', header=None, sep=' ')

In [ ]:
balanced_female_pairs = pd.concat([female_pairs[female_pairs[0] == 1].reset_index(drop=True), female_pairs[female_pairs[0] == 0].sample(sum(female_pairs[0] == 1)).reset_index(drop=True)]).sort_index().reset_index(drop=True)

In [ ]:
balanced_female_pairs

In [ ]:
balanced_male_pairs.to_csv('/home/jupyter/voxceleb-fairness/data/lists/vox1_male_all_balanced.txt', index=False, header=None, sep=' ')

In [ ]:
balanced_female_pairs.to_csv('/home/jupyter/voxceleb-fairness/data/lists/vox1_female_all_balanced.txt', index=False, header=None, sep=' ')